# 《心境流转》系统快速初始化

本notebook是专为JupyterHub环境设计的快速初始化脚本。

## 🎯 目标
- 快速验证JupyterHub环境
- 自动创建所需的系统模块
- 确保系统能够正常运行

In [ ]:
# 1. 环境设置和项目结构创建
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("=" * 50)
print("《心境流转》快速初始化")
print("=" * 50)

# 项目根目录检测
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    PROJECT_ROOT = current_dir.parent
elif (current_dir / 'notebooks').exists():
    PROJECT_ROOT = current_dir
else:
    PROJECT_ROOT = current_dir
    for parent in current_dir.parents:
        if (parent / 'notebooks').exists():
            PROJECT_ROOT = parent
            break

print(f"📂 项目根目录: {PROJECT_ROOT}")

# 添加Python路径
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / 'src'))
sys.path.insert(0, str(PROJECT_ROOT / 'research'))

print("✅ Python路径配置完成")

In [ ]:
# 2. 创建项目目录结构
print("\n📁 创建项目结构...")

directories = [
    'src', 'src/core', 'src/models', 'src/therapy', 'src/optimization', 'src/evaluation',
    'research', 'research/theory', 'outputs', 'outputs/cache', 'configs', 'api'
]

for directory in directories:
    dir_path = PROJECT_ROOT / directory
    dir_path.mkdir(parents=True, exist_ok=True)
    
    if directory.startswith(('src/', 'research/')):
        init_file = dir_path / '__init__.py'
        if not init_file.exists():
            init_file.write_text('"""模块初始化"""\n')

print("✅ 项目结构创建完成")

In [ ]:
# 3. 快速硬件检测
import platform
from datetime import datetime

print("\n🖥️ 系统信息:")
print(f"Python: {sys.version.split()[0]}")
print(f"平台: {platform.system()}")
print(f"时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# GPU检测
gpu_info = "CPU模式"
try:
    import torch
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        props = torch.cuda.get_device_properties(0)
        memory_gb = props.total_memory / (1024**3)
        gpu_info = f"GPU: {props.name} ({memory_gb:.1f}GB)"
    print(f"🎮 {gpu_info}")
except:
    print(f"💻 {gpu_info}")

print("✅ 硬件检测完成")

In [ ]:
# 4. 创建理论模块
print("\n📚 创建理论模块...")

# ISO原则模块
iso_code = '''"""ISO三阶段治疗原则"""
from dataclasses import dataclass

@dataclass
class EmotionState:
    valence: float
    arousal: float
    confidence: float = 0.8

class ISOPrinciple:
    def __init__(self):
        self.stages = ['synchronization', 'guidance', 'consolidation']
        print("🎵 ISO三阶段治疗原则已初始化")
    
    def plan_therapy_stages(self, current_emotion, target_emotion):
        return [
            {'stage': 'sync', 'duration': 10},
            {'stage': 'guide', 'duration': 15},
            {'stage': 'consolidate', 'duration': 5}
        ]
'''

# VA模型模块
va_code = '''"""情绪价值-唤醒模型"""
class ValenceArousalModel:
    def __init__(self):
        self.emotion_map = {
            'happy': (0.8, 0.7), 'calm': (0.5, 0.2),
            'sad': (-0.6, 0.3), 'angry': (-0.3, 0.8)
        }
        print("📊 V-A情绪模型已初始化")
    
    def get_emotion_coordinates(self, emotion):
        return self.emotion_map.get(emotion, (0.0, 0.5))
'''

# 写入文件
(PROJECT_ROOT / 'research' / 'theory' / 'iso_principle.py').write_text(iso_code)
(PROJECT_ROOT / 'research' / 'theory' / 'valence_arousal.py').write_text(va_code)

print("✅ 理论模块创建完成")

In [ ]:
# 5. 创建模型适配器
print("\n🤖 创建模型适配器...")

# 基础适配器
base_code = '''"""基础模型适配器"""
from dataclasses import dataclass

@dataclass
class ModelConfig:
    model_name: str
    model_type: str
    device: str = "cpu"

class BaseModelAdapter:
    def __init__(self, config):
        self.config = config
        self.is_loaded = False
    
    def load_model(self):
        self.is_loaded = True
        return True
    
    def predict(self, input_data):
        return {"result": "mock_prediction", "confidence": 0.85}
'''

# 模型工厂
factory_code = '''"""模型工厂"""
from .base import BaseModelAdapter, ModelConfig

class ModelFactory:
    def __init__(self):
        self.models = {
            'emotion_text': 'roberta-emotion',
            'emotion_audio': 'wav2vec2-emotion',
            'music_generation': 'musicgen-small'
        }
        print("🏭 模型工厂已初始化")
    
    def create_model_adapter(self, model_type):
        model_name = self.models.get(model_type, f"mock_{model_type}")
        config = ModelConfig(model_name=model_name, model_type=model_type)
        return BaseModelAdapter(config)
    
    def get_recommended_models(self):
        return {"profile": "gpu_40gb", "recommended": self.models}
'''

# 写入文件
(PROJECT_ROOT / 'src' / 'models' / 'base.py').write_text(base_code)
(PROJECT_ROOT / 'src' / 'models' / 'factory.py').write_text(factory_code)

print("✅ 模型适配器创建完成")

In [ ]:
# 6. 创建疗愈系统
print("\n🧘 创建疗愈系统...")

therapy_code = '''"""疗愈系统核心"""
from datetime import datetime
import uuid

class TherapySession:
    def __init__(self, user_id, current_emotion, target_emotion):
        self.session_id = str(uuid.uuid4())[:8]
        self.user_id = user_id
        self.start_time = datetime.now()
        self.current_emotion = current_emotion
        self.target_emotion = target_emotion
        self.status = "active"

class TherapyOrchestrator:
    def __init__(self):
        self.active_sessions = {}
        print("🧘 疗愈编排器已初始化")
    
    def create_session(self, user_id, current_emotion, target_emotion):
        session = TherapySession(user_id, current_emotion, target_emotion)
        self.active_sessions[session.session_id] = session
        print(f"🎯 创建疗愈会话: {session.session_id}")
        return session
    
    def get_session_recommendations(self, session_id):
        return {
            "session_id": session_id,
            "recommendations": {
                "music_style": "ambient",
                "tempo_bpm": 60,
                "duration_minutes": 30
            }
        }

class PrescriptionEngine:
    def __init__(self):
        print("💊 处方引擎已初始化")
    
    def generate_prescription(self, user_profile, current_emotion, target_emotion):
        return {
            "prescription_id": f"rx_{hash(str(current_emotion)) % 1000:03d}",
            "recommendations": {
                "music_genre": "ambient",
                "tempo_bpm": 60,
                "duration_minutes": 25
            },
            "confidence_score": 0.85
        }
'''

# 写入文件
(PROJECT_ROOT / 'src' / 'therapy' / 'core.py').write_text(therapy_code)

print("✅ 疗愈系统创建完成")

In [ ]:
# 7. 测试所有模块
print("\n🧪 测试系统模块...")

# 清理模块缓存
modules_to_clear = [name for name in sys.modules.keys() 
                   if name.startswith(('research.', 'src.'))]
for module_name in modules_to_clear:
    if module_name in sys.modules:
        del sys.modules[module_name]

success_count = 0
total_tests = 3

# 测试理论模块
try:
    from research.theory.iso_principle import ISOPrinciple, EmotionState
    from research.theory.valence_arousal import ValenceArousalModel
    
    emotion_state = EmotionState(valence=-0.2, arousal=0.6)
    iso_planner = ISOPrinciple()
    va_model = ValenceArousalModel()
    
    print("✅ 理论模块测试通过")
    success_count += 1
except Exception as e:
    print(f"❌ 理论模块测试失败: {e}")

# 测试模型适配器
try:
    from src.models.factory import ModelFactory
    from src.models.base import BaseModelAdapter
    
    factory = ModelFactory()
    adapter = factory.create_model_adapter('emotion_text')
    recommendations = factory.get_recommended_models()
    
    print(f"✅ 模型适配器测试通过 ({recommendations['profile']})")
    success_count += 1
except Exception as e:
    print(f"❌ 模型适配器测试失败: {e}")

# 测试疗愈系统
try:
    from src.therapy.core import TherapyOrchestrator, PrescriptionEngine
    
    orchestrator = TherapyOrchestrator()
    prescription_engine = PrescriptionEngine()
    
    session = orchestrator.create_session(
        "test_user",
        {"valence": -0.4, "arousal": 0.8},
        {"valence": 0.6, "arousal": 0.3}
    )
    
    prescription = prescription_engine.generate_prescription(
        {}, {"valence": -0.4, "arousal": 0.8}, {"valence": 0.6, "arousal": 0.3}
    )
    
    print(f"✅ 疗愈系统测试通过 (会话: {session.session_id})")
    success_count += 1
except Exception as e:
    print(f"❌ 疗愈系统测试失败: {e}")

print(f"\n📊 测试结果: {success_count}/{total_tests} 通过")

In [ ]:
# 8. 保存状态和总结
import json

# 系统状态
system_status = {
    "initialization_time": datetime.now().isoformat(),
    "python_version": sys.version.split()[0],
    "platform": platform.system(),
    "project_root": str(PROJECT_ROOT),
    "tests_passed": success_count,
    "total_tests": total_tests,
    "success_rate": success_count / total_tests,
    "status": "success" if success_count == total_tests else "partial"
}

# 保存状态文件
status_file = PROJECT_ROOT / 'outputs' / 'quick_init_status.json'
with open(status_file, 'w', encoding='utf-8') as f:
    json.dump(system_status, f, indent=2, ensure_ascii=False)

# 显示最终结果
print("\n" + "=" * 50)
print("《心境流转》快速初始化完成")
print("=" * 50)

if success_count == total_tests:
    print("🎉 所有模块初始化成功！")
    print("\n💡 下一步:")
    print("  运行其他测试notebook继续验证系统功能")
else:
    print(f"⚠️  {success_count}/{total_tests} 模块成功，{total_tests - success_count} 模块需要检查")

print(f"\n📊 初始化信息:")
print(f"  项目路径: {PROJECT_ROOT}")
print(f"  成功率: {success_count}/{total_tests} ({success_count/total_tests*100:.0f}%)")
print(f"  状态文件: {status_file}")
print(f"  完成时间: {datetime.now().strftime('%H:%M:%S')}")

print("\n✅ 快速初始化完成！系统可以使用了")